In [ ]:
import os
import sys
import anndata
import pandas as pd
import numpy as np

from data_processing.data_loader import DataLoader
from data_processing.split_cell import split_cell
from data_processing.preprocess import Preprocess_gene
from data_processing.data import TwitterDataset
from AlignCell.train import training
from AlignCell.run_model_training import run_model_training

In [ ]:
#Input the gene expression matrix with gene names as row names and cell names as column names.
matrix_1 = pd.read_csv('./example/testis_merge_matrix.csv',index_col=0)
matrix_1

,Donor2-AAACCTGGTGCCTTGG-1,Donor2-AAACCTGTCAACGGGA-1,Donor2-AAACCTGTCCTATGTT-1,Donor2-AAACCTGTCGGACAAG-1,Donor2-AAACGGGAGCTATGCT-1,Donor2-AAACGGGGTCTGGAGA-1,Donor2-AAACGGGTCTCTAAGG-1,Donor2-AAATGCCAGTGTTTGC-1,Donor2-AAATGCCGTTAGTGGG-1,Donor2-AAATGCCGTTCGTCTC-1,...,N3_TESTIS_10_28,N3_TESTIS_10_29,N3_TESTIS_10_33,N3_TESTIS_10_47,N3_TESTIS_11_20,N3_TESTIS_11_38,N3_TESTIS_12_14,N3_TESTIS_12_35,N3_TESTIS_12_37,1_SY_23
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00
A1BG-AS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00
A1CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00
A2M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00
A2M-AS1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,20.77,0.00,47.29,0.0,0.00,0.0,0.00
ZYX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,21.97,20.77,17.54,0.00,0.0,74.66,0.0,0.00
ZZEF1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.00,20.77,0.00,0.00,0.0,0.00,0.0,72.65


In [3]:
fil_type1=pd.read_csv('./example/testis_merge_type.csv')
fil_type1

,Batch,Cell_Name,Type
0,10X,Donor2-AAACCTGGTGCCTTGG-1,Elongated S'tids
1,10X,Donor2-AAACCTGTCAACGGGA-1,Sperm
2,10X,Donor2-AAACCTGTCCTATGTT-1,Sperm
3,10X,Donor2-AAACCTGTCGGACAAG-1,Sperm
4,10X,Donor2-AAACGGGAGCTATGCT-1,Sperm
...,...,...,...
9339,SS2,N3_TESTIS_11_38,Sperm
9340,SS2,N3_TESTIS_12_14,Sperm
9341,SS2,N3_TESTIS_12_35,Sperm
9342,SS2,N3_TESTIS_12_37,Sperm


In [ ]:
#Select one of the sources as the training set.
obs_df = fil_type1[fil_type1['Batch'] == '10X']
expression_df=matrix_1.loc[:, matrix_1.columns.isin(obs_df['Cell_Name'])]

In [ ]:
#This function allows splitting the dataset into training and test sets based on the "type" attribute. 
#In this case, we are simply collecting one batch as the training set, so the train_ratio parameter is set to 1. 
# his step can actually be omitted unless you need to use the "type" information to divide the training and test sets.
train_data, test_data, train_expression_matrix, test_expression_matrix = split_cell(obs_df, expression_df, 
                                                                                    group_column='Type', cell_name_column='Cell_Name',
                                                                                    random_state=9, train_ratio=1)

In [ ]:
#Process the metadata and the corresponding data.
loader1 = DataLoader(expression_matrix_1=train_expression_matrix, cell_labels_1=train_data, top_genes=1000, mode="train",cell_name_column='Cell_Name',
                     cell_type_column='Type')

ref_dat, ref_label, ref_cell_labels = loader1.process_data()

print("Updated ref_cell_labels:")
print(ref_cell_labels)

更新后的 ref_cell_labels:
     Batch                  Cell_Name                     Type cell_id   mode  \
0      10X  Donor2-GAACCTACAGCAGTTT-2  Differentiating S'gonia     3_1  train   
1      10X  Donor3-ATCTACTTCTCCAACC-2  Differentiating S'gonia     3_2  train   
2      10X  Donor3-CGGCTAGTCGGTTCGG-2  Differentiating S'gonia     3_3  train   
3      10X  Donor1-TGGCCAGAGCTACCGC-2  Differentiating S'gonia     3_4  train   
4      10X  Donor3-CAGCAGCGTCTTTCAT-2  Differentiating S'gonia     3_5  train   
...    ...                        ...                      ...     ...    ...   
6485   10X  Donor2-ATAACGCTCTACCAGA-1                    Sperm  3_6486  train   
6486   10X  Donor2-GGTGTTAAGAGTACCG-2                    Sperm  3_6487  train   
6487   10X  Donor3-ACTGAACCAGCGTAAG-1                    Sperm  3_6488  train   
6488   10X  Donor2-ATTCTACAGATAGGAG-2                    Sperm  3_6489  train   
6489   10X  Donor2-CCACGGATCTCCGGTT-2                    Sperm  3_6490  train   

     

In [ ]:
#Encode genes using the trained w2v model. The default path uses the trained model.
#If you have your own trained w2v vectors, you can add them manually—make sure the version number matches.
preprocess = Preprocess_gene(ref_dat, sen_len=1000)
embedding = preprocess.make_embedding()
gene_name = preprocess.sentence_word2idx()
gene_label = preprocess.labels_to_tensor(ref_label)

Get embedding ...
loading word to vec model ...
get words #84312
total words: 84315


In [ ]:
#The mode parameter has two options:
#Use 'positive_negative' mode when training the model to construct triplet loss sample pairs.
#Use 'single' mode when applying the trained model for prediction.
ref_dataset = TwitterDataset(x=gene_name, z=gene_label, mode="positive_negative")

In [9]:
from get_result.Get_emb import AlignCell_Emb
from AlignCell import TransformerEncoder, PositionalEncoding, EncoderLayer, FeedForward

In [ ]:
#Start training
run_model_training(
    embedding=embedding,
    training=training,
    all_dataset=ref_dataset,
    pretrained_model_path="./pretrain_model/ckpt_hum_cell_pretrain.model",#This parameter loads a pre-trained model. If not provided, a new model will be created.
    model_dir="./new_testis",
    epochs=200
)

Loading pretrained model from ../ckpt_hum_cell_pretrain.model...
Starting training...

start training, parameter total:26249840, trainable:26249840

start1

start2

